In [ ]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

# loading the resnet model/*include_top=False: When include_top is set to False, it means you don't want to include the final classification layer of the ResNet50 model. Instead, you intend to use the ResNet50 model as a feature extractor, which will provide you with feature maps for input images.*/
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3)) #224,224,3 standard size of an image as per transfer learning
model.trainable = False

model = tensorflow.keras.Sequential([        # processing the resNET model upto our standards that is puting top layer with GlobalMaxPooling2D()
    model,
    GlobalMaxPooling2D()
])

In [2]:
import numpy as np
from numpy.linalg import norm #use in normalization(feature scaling)
import os

def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224)) # loading image
    img_array = image.img_to_array(img) # image to numpy array to work upon the array
    expanded_img_array = np.expand_dims(img_array, axis=0) # expanding the image to batches for keras to act upon as keras works on batches of inage and not a single image
    preprocessed_img = preprocess_input(expanded_img_array)# preprocessed_input is a fucnction under resNET model and it extacts features of an image in [1, 2048] format
    result = model.predict(preprocessed_img).flatten() # converting 2d processed_img to 1d by flattening it that is [,2048]
    normalized_result = result / norm(result) # feature scaling using normalization so that reslut variable stores values between 0 to 1

    return normalized_result

In [3]:
from tqdm import tqdm # track progress of for loop
import pickle

# list to store filename of the images in the dataset
filenames = []
for file in os.listdir('images'):
    filenames.append(os.path.join('images',file))

# 2d list to store features of the images in the dataset by passing its filename to extract_feature function and resnetmodel
feature_list = []
for file in tqdm(filenames):
    feature_list.append(extract_features(file,model)) # [[1,2048],[1,2048].....,[1,2048]]

# storing above gotten outputs as file i the directory to use it in further implementations conviniently
pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))


100%|██████████| 2228/2228 [08:21<00:00,  4.45it/s]


In [8]:
# FOR THE INPUT IMAGE
import pickle
import tensorflow
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2
# loading the files created above with features and filename of images stored
feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))
# model loading again for the user input image
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])
#sample contains an image of every type in the dataset so that we get the vector through reset of that image. Limitation: input must be from the sample
def switch(val):
    return{
    "jersey": image.load_img('sample/jersey.jpg',target_size=(224,224)),
        
    "watch": image.load_img('sample/watch.jpg',target_size=(224,224)),
        
    "trouser": image.load_img('sample/trouser.jpg',target_size=(224,224)),
        
    "short": image.load_img('sample/short.jpg',target_size=(224,224)),
        
    "top": image.load_img('sample/top.jpg',target_size=(224,224)),
        
    "skirt": image.load_img('sample/skirt.jpg',target_size=(224,224)),
        
       "shoe": image.load_img('sample/shoe.jpg',target_size=(224,224)),
        
    "sock": image.load_img('sample/sock.jpg',target_size=(224,224)),
    }.get(val,"nothing")

val = str(input("Enter your search: "))
img = switch(val)
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)
# finding the nearest neighbours using the used function and algo brute force as there s not much data and 6 cuz first is the image itself and euclidean as cosine distance is less optimal
neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

distances,indices = neighbors.kneighbors([normalized_result]) # getting the indices of the nearest neighbours of the input image from the normalized_result list


for file in indices[0][1:6]: 
    temp_img = cv2.imread(filenames[file]) # cv2 to display the image by passing the index of the nearest neighbout to filename and get the filename to display
    cv2.imshow('output',cv2.resize(temp_img,(512,512))) # output name of the display window
    cv2.waitKey(0) # waittype bnetween every pop up

KeyboardInterrupt: Interrupted by user